In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import timm
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models.segmentation as models
from datasets.TrainDataset import TrainDataset
import os
import random
import numpy as np
import torch
import matplotlib as plt
from examples.mask_to_submission import *
from datasets.TestDataset import TestDataset

/home/stef/anaconda3/envs/road_segmentation/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# set seeds as function
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)


# set seeds
set_seed()

# create folder models if it doesn't exist
if not os.path.exists("models"):
    os.makedirs("models")

In [4]:
# define data transformation
transform = transforms.Compose(
    [
        transforms.Resize((400, 400)),  # crashes with 400 x 400
        transforms.ToTensor(),
        # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [5]:
# paths to image and ground truth folders
image_folder = "datasets/train/images/"
gt_folder = "datasets/train/groundtruth/"

In [6]:
# create an instance of custom dataset class
dataset = TrainDataset(image_folder, gt_folder, transform=transform)

In [7]:
# seed for reproducibility
torch.manual_seed(0)
# split the dataset into training and validation sets
train_set, val_set = torch.utils.data.random_split(
    dataset, [int(0.8 * len(dataset)), int(0.2 * len(dataset))]
)

In [8]:
# define batch size
batch_size = 2
# create data loaders
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True)

In [10]:
# pre-trained DeepLabV3 model
model = models.deeplabv3_resnet101(num_classes=1, pretrained=False)

/home/stef/anaconda3/envs/road_segmentation/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/stef/anaconda3/envs/road_segmentation/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [11]:
# define loss function
criterion = torch.nn.BCEWithLogitsLoss()

In [12]:
# define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
# number of epochs to train the model
n_epochs = 50
# device to use for training
# check device
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_built()
    else "cpu"
)
print(device)
model.to(device)

cuda


DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [14]:
# Assuming the following are already defined:
# model, criterion, optimizer, train_loader, val_loader, device

n_epochs = 30  # Set the number of epochs

# Initialize a variable to track the minimum validation loss
best_val_loss = float("inf")

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    for images, masks in train_loader:  # Assuming you have a train_loader
        images, masks = images.to(device), masks.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs["out"], masks.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs["out"], labels.float())
            val_loss += loss.item()

    # calculate loss
    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)

    print(
        f"Epoch: {epoch + 1} | Training Loss: {avg_train_loss:.4f} | Validation Loss: {avg_val_loss:.4f}"
    )

    # best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        # Save the model
        torch.save(model.state_dict(), "models/deeplabv3_resnet101.pt")
        print(f"Model saved at epoch {epoch + 1}")

Epoch: 1 | Training Loss: 0.4697 | Validation Loss: 0.5175
Model saved at epoch 1
Epoch: 2 | Training Loss: 0.2690 | Validation Loss: 0.3198
Model saved at epoch 2
Epoch: 3 | Training Loss: 0.2488 | Validation Loss: 0.2387
Model saved at epoch 3
Epoch: 4 | Training Loss: 0.2189 | Validation Loss: 0.2158
Model saved at epoch 4
Epoch: 5 | Training Loss: 0.1992 | Validation Loss: 0.2174
Epoch: 6 | Training Loss: 0.1820 | Validation Loss: 0.2379
Epoch: 7 | Training Loss: 0.1679 | Validation Loss: 0.2323
Epoch: 8 | Training Loss: 0.1618 | Validation Loss: 0.1913
Model saved at epoch 8
Epoch: 9 | Training Loss: 0.1504 | Validation Loss: 0.2237
Epoch: 10 | Training Loss: 0.1618 | Validation Loss: 0.1961
Epoch: 11 | Training Loss: 0.1516 | Validation Loss: 0.2175
Epoch: 12 | Training Loss: 0.1439 | Validation Loss: 0.1934
Epoch: 13 | Training Loss: 0.1319 | Validation Loss: 0.2822
Epoch: 14 | Training Loss: 0.1342 | Validation Loss: 0.1678
Model saved at epoch 14
Epoch: 15 | Training Loss: 0.1

Submission

In [ ]:
MODEL = "models/unet.pt"

In [ ]:
MODEL = "models/deeplabv3_resnet101.pt"

model = models.deeplabv3_resnet101(num_classes=1, pretrained=False)
# Replace YOUR_NUM_CLASSES with the number of classes in your specific case

# Load the state dictionary
model.load_state_dict(torch.load("models/deeplabv3_resnet101.pt"))

model.eval()

/home/stef/anaconda3/envs/road_segmentation/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/stef/anaconda3/envs/road_segmentation/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [ ]:
# path to the test folder
test_folder = "datasets/test/"

# define transformations
transform = transforms.Compose([transforms.ToTensor()])

# create test dataset
test_dataset = TestDataset(test_folder, transform=transform)

['datasets/test/test_1/test_1.png', 'datasets/test/test_2/test_2.png', 'datasets/test/test_3/test_3.png', 'datasets/test/test_4/test_4.png', 'datasets/test/test_5/test_5.png', 'datasets/test/test_6/test_6.png', 'datasets/test/test_7/test_7.png', 'datasets/test/test_8/test_8.png', 'datasets/test/test_9/test_9.png', 'datasets/test/test_10/test_10.png', 'datasets/test/test_11/test_11.png', 'datasets/test/test_12/test_12.png', 'datasets/test/test_13/test_13.png', 'datasets/test/test_14/test_14.png', 'datasets/test/test_15/test_15.png', 'datasets/test/test_16/test_16.png', 'datasets/test/test_17/test_17.png', 'datasets/test/test_18/test_18.png', 'datasets/test/test_19/test_19.png', 'datasets/test/test_20/test_20.png', 'datasets/test/test_21/test_21.png', 'datasets/test/test_22/test_22.png', 'datasets/test/test_23/test_23.png', 'datasets/test/test_24/test_24.png', 'datasets/test/test_25/test_25.png', 'datasets/test/test_26/test_26.png', 'datasets/test/test_27/test_27.png', 'datasets/test/tes

In [ ]:
# save predictions
prediction_filenames = []
for i in range(len(test_dataset)):
    # get image
    image = test_dataset[i]
    # create prediction
    prediction = model(image.unsqueeze(0))
    prediction = prediction["out"]
    # threshold prediction
    prediction = (prediction > 0.5).float()
    # save prediction
    prediction_filename = "predictions/prediction_" + str(i + 1) + ".png"
    prediction_filenames.append(prediction_filename)
    plt.imsave(prediction_filename, prediction.squeeze().detach().numpy(), cmap="gray")

In [ ]:
# create submission
masks_to_submission("submission.csv", *prediction_filenames)